# Checking Moments of Gaussian Random Field
This shows how we can sample a Gaussian random variable by FFT methods.  In this excample, the mean is zero, and the covariance is
$$
C = (-d^2/dx^2)^{-1} + \text{Dirichlet Boundary Conditions}
$$
The eigenvalue/eigenfunction pairs are
$$
\lambda_k = \frac{1}{(\pi k)^2}, \quad \varphi_k(x) = \sqrt{2}\sin(\pi k x)
$$

By deriving a Green's function for $C$, it can be derived that $\mathbb{E}[X(x)] = 0$ and
$$
\mathbb{E}[X(x)^2] = x(1-x).
$$
Here, we will verify that we are properly sampling these moments.

In [ ]:
using Plots; pyplot()
using Random
using LaTeXStrings
using Statistics

In [ ]:
using GaussianRV

In [ ]:
default(xtickfont=font(14),  ytickfont=font(14), guidefont=font(14), 
    legendfontsize=12, lw=2,ms=8)

In [ ]:
N = 256; # set number of spatial points/KLSE modes
n_samples = 10^4; # number of independent samples
x = LinRange(0,1,N+1)[2:end-1];

u_samples = zeros(N-1, n_samples);

In [ ]:
Random.seed!(100)
Threads.@threads for j in 1:n_samples
    u_samples[:,j] .= sample_field(N);
end

## Compare the Moments

In [ ]:
Eu = mean(u_samples, dims=2);
Eu2 = mean(u_samples.^2, dims=2);

plot(x, Eu,label=L"Estimated $E[X]$",legend=:left)
plot!(x, Eu2,label=L"Estimated $E[X^2]$")
plot!(x, 0*x,label=L"$E[X]$")
plot!(x, (@. x*(1-x)),label=L"$E[X^2]$")
xlabel!(L"$x$")

## Plot with Error Bars
This checks the estimate of the first moment, as a function of $x$, and a 95\% Gaussian confidence interval.

In [ ]:
plot(x, Eu, ribbon=1.96* std(u_samples,dims=2)/sqrt(n_samples), 
    label=L"Estimated $E[X]$",legend=:topleft)
plot!(x, 0*x,label=L"$E[X]$")
xlabel!(L"$x$")